In [1]:
import pandas as pd
df=pd.read_csv('mon_data.csv')
df.dropna(subset=['text_image'],inplace=True)

In [2]:
df=df.iloc[:,[-1,3]]

In [3]:
"""from nltk.corpus import stopwords
from nltk.tokenize.regexp import  RegexpTokenizer
stop_words = set(stopwords.words('french'))
stop_words.update(["." , ",",':'])
tokenizer=RegexpTokenizer(("[a-zA-Zé]{4,}"))
def stop_words_filtering(liste):
    liste_new=[]
    for i in liste:
        if i not in stop_words:
            liste_new.append(i)
        else:
            continue 
    return liste_new
sentences=[]
for txt in df.text_image:
    mots=stop_words_filtering(tokenizer.tokenize(str(txt).lower()))
    sentences.append(mots)"""

In [3]:
import re
import unicodedata
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english'))
stop_words.update(["." , ",",':'])
stop_words=list(stop_words)
stop_words.extend(stopwords.words('french'))
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')

def preprocess_sentence(w):
    w = unicode_to_ascii(str(w).lower().strip())
    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    w = re.sub(r"[^a-zA-Z?.!]+", " ", w)
    w = re.sub(r'\b\w{0,2}\b', '', w)

    # remove stopword
    mots = word_tokenize(w.strip())
    mots = [mot for mot in mots if mot not in stop_words]
    return ' '.join(mots).strip()

In [4]:
df.text_image = df.text_image.apply(lambda x :preprocess_sentence(x))

In [5]:
import gensim
docs=[]
for i, text in enumerate(df.text_image):
    words = text.split()
    tags = [i]
    docs.append(gensim.models.doc2vec.TaggedDocument(words,tags))
    

In [51]:
docs

[TaggedDocument(words=['facture', 'logo', 'joanna', 'binet', 'coubertin', 'paris', 'facture', 'cendrillon', 'ayot', 'rue', 'nations', 'paris', 'envoye', 'cendrillon', 'ayot', 'rue', 'ferreol', 'lle', 'france', 'facture', 'date', 'commande', 'echeance', 'prix', 'unit', '.', '.', '.', '.', 'total', 'tva', '.', 'total', 'montant', '.', '.', '.', '.', '.', '.', 'qte', 'designation', 'grand', 'brun', 'escargot', 'pour', 'manger', 'petit', 'mariniere', 'uniforme', 'bleu', 'facile', 'jouer', 'accordeon', 'buncht', 'conditions', 'modalites', 'paiement', 'paiement', 'est', 'dans', 'jours', 'caisse', 'epargne', 'iban', 'swift', 'bic', 'abcdfrp', 'xxx', 'merci'], tags=[0]),
 TaggedDocument(words=['joanna', 'binet', 'coubertin', 'paris', 'facture', 'facture', 'cendrillon', 'ayot', 'rue', 'nations', 'paris', 'envoye', 'cendrillon', 'ayot', 'rue', 'ferreol', 'ile', 'france', 'facture', 'date', 'commande', 'echeance', 'qte', 'designation', 'prix', 'unit', '.', 'montant', 'grand', 'brun', 'escargot', 

In [84]:
from gensim.models import Doc2Vec
max_epochs = 100 
vec_size = 100 
alpha = 0.025 

model = Doc2Vec(#size=vec_size, 
                alpha=alpha, 
                min_alpha=0.00025, 
                min_count=1, 
                dm =1) 

In [85]:
model.build_vocab(docs)


In [86]:
model.train(docs,total_examples=model.corpus_count, epochs=100)

In [87]:
total_text=[]
for i, text in enumerate(df.text_image):
    words_lis = text.split()
    total_text.append(model.infer_vector(words_lis))

In [88]:
total_donnes=pd.DataFrame(np.array(total_text))
total_donnes['type']=df.type

In [93]:
total_donnes.shape
total_donnes.dropna(inplace=True)
total_donnes.head()

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,type
0,-0.271355,0.299643,-0.131775,-1.825927,0.983947,-0.667552,0.164891,0.857895,-0.755930,0.365526,...,0.302167,0.211270,0.077230,0.609552,0.092000,0.392901,0.325528,0.062940,-0.318427,0.0
1,-0.011222,0.541586,-0.020482,-1.403974,0.857083,-0.601009,-0.118539,0.776590,-0.829467,0.397217,...,0.147566,-0.016586,0.089250,0.612785,-0.173147,0.341955,0.212249,-0.138698,-0.316857,0.0
2,0.526088,0.578944,0.185538,-0.901332,0.131467,-0.642516,-0.030721,0.378258,-0.249853,-0.009083,...,0.657348,0.027401,-0.184171,0.236673,-0.239735,-0.041332,0.113330,-0.271099,-0.274059,0.0
3,-0.135574,0.597531,-0.107510,-1.623078,0.757028,-0.851101,0.030996,0.748700,-0.863840,0.213517,...,0.175278,-0.070680,0.096822,0.629425,-0.078992,0.376727,0.080292,-0.352869,-0.215665,0.0
4,0.004755,0.792993,-0.105368,-1.218495,0.391444,-0.658981,-0.062820,0.562467,-1.211365,0.015313,...,0.791595,0.205286,-0.081631,0.819581,-0.103569,0.009150,0.029504,-0.065222,-0.620588,0.0


In [94]:
data=total_donnes.drop('type',axis=1)

target=total_donnes.type

In [95]:
from sklearn.model_selection import train_test_split
X_train,X_test, y_train, y_test=train_test_split(data,target,test_size=0.2,random_state=1234)

In [96]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, Dense, GlobalAveragePooling1D, Dropout,Activation
model=Sequential()
model.add(Dense(units = 256,input_shape=(100,)))
model.add((Activation('relu')))
model.add(Dropout(0.3))
model.add(Dense(units = 23, activation = "softmax"))
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 256)               25856     
                                                                 
 activation_4 (Activation)   (None, 256)               0         
                                                                 
 dropout_4 (Dropout)         (None, 256)               0         
                                                                 
 dense_9 (Dense)             (None, 23)                5911      
                                                                 
Total params: 31,767
Trainable params: 31,767
Non-trainable params: 0
_________________________________________________________________


In [97]:
model.compile(loss="sparse_categorical_crossentropy",optimizer = "adam", metrics = ["accuracy"])
model.fit(X_train,y_train,epochs = 50, batch_size = 32,validation_data = [X_test, y_test])

Epoch 1/50
39/39 [==============================] - 1s 8ms/step - loss: 3.1017 - accuracy: 0.2108 - val_loss: 2.7459 - val_accuracy: 0.3115
Epoch 2/50
39/39 [==============================] - 0s 3ms/step - loss: 2.5719 - accuracy: 0.3585 - val_loss: 2.5933 - val_accuracy: 0.3770
Epoch 3/50
39/39 [==============================] - 0s 3ms/step - loss: 2.3138 - accuracy: 0.4110 - val_loss: 2.5136 - val_accuracy: 0.3869
Epoch 4/50
39/39 [==============================] - 0s 3ms/step - loss: 2.1403 - accuracy: 0.4545 - val_loss: 2.4180 - val_accuracy: 0.3902
Epoch 5/50
39/39 [==============================] - 0s 3ms/step - loss: 1.9607 - accuracy: 0.4881 - val_loss: 2.3990 - val_accuracy: 0.4164
Epoch 6/50
39/39 [==============================] - 0s 3ms/step - loss: 1.8402 - accuracy: 0.5357 - val_loss: 2.3671 - val_accuracy: 0.4230
Epoch 7/50
39/39 [==============================] - 0s 3ms/step - loss: 1.7605 - accuracy: 0.5332 - val_loss: 2.3600 - val_accuracy: 0.4295
Epoch 8/50
39/39 [==

In [98]:
model.evaluate(X_test, y_test)

10/10 [==============================] - 0s 2ms/step - loss: 3.0593 - accuracy: 0.4197


[3.0593018531799316, 0.4196721315383911]